# Cytoscape Reactome FI Example

Overview
--------
This example demonstrates a workflow using ReactomeFIViz CyREST API to perform a comparison pathway and network analysis for TCGA BRCA (breast invasive carcinoma) and OV (ovarian serous cystadenocarcinoma) mutation data.

The steps executed are as follows:

1. Download each cohort mutation annotation file (MAF) from the Broad Firehose WS server.

2. Build the two networks using the ReactomeFIViz CyREST API buildFISubNetwork resource.

3. Cluster the network into network modules using the cluster resource.

4. Perform pair-wise module overlapping analysis.

5. Load the Reactome pathways.

6. Partition the network modules generated from two networks into the following sets:

  * Modules with a significant proportion of genes in common

  * Unshared modules for each of the two cancer types

7. Perform pathway enrichment analysis for each unshared gene module.

8. Export the diagram for an unshared pathway for each cancer type.

Prerequisites
------------
* Python version 2.6 or later

* Cytoscape with the latest version of ReactomeFIViz

Install the required packages in a Python environment, e.g. in the
[Anaconda](https://docs.anaconda.com/anaconda/) virtual environment:

    conda create -n cytoscape pip pandas scipy statsmodels
    source activate cytoscape

Open Cytoscape before building the networks below. Select `Help > Check for Updates`
to ensure that ReactomeFIViz is current.

*Note*: Building the networks clears an existing Cytoscape session, if it exists.

In [ ]:
# Python utilities.
import sys
import os
import requests
import tempfile
import shutil
import itertools
from io import StringIO
import pandas as pd
from scipy.stats import hypergeom
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests
if sys.version_info < (3,0):
    from __future__ import print_function

In [ ]:
# Create the MAF download area.
# The try-except idiom below, although usually discouraged,
# avoids creating a new sandbox for repeated executions.
try:
    sandbox
except NameError:
    sandbox = tempfile.mkdtemp()
print("The downloaded MAF files will be saved to %s." % sandbox)

In [ ]:
# The REST service url.
SERVICE_URL = 'http://localhost:1234'

def get_url(*params, **kwargs):
    """
    :param params: the URL path component strings
    :option app: the CyREST application name
    :return: the REST app URL
    """
    path = [SERVICE_URL]
    app = kwargs.get('app')
    if app is not None:
        path.append(app)
    path.append('v1')
    path.extend(params)
    return '/'.join(path)

# Returns the CyREST Reactome FI url.
get_fi_url = lambda *params: get_url(*params, app='reactomefiviz')

In [ ]:
def parse_fi_table_response(resp, parsers, index=None):
    """
    Returns the data frame for the given CyREST Reactome
    FI response.  The response must contain a `data`
    property object with content:
    
        {tableHeaders, tableHeaders, tableContent}.
    
    The required *parsers* dictionary argument associates
    a parser with each column.
    
    :param resp: the CyREST response
    :param parsers: the column parser dictionary
    :option index: the index column name
"""
    # The cluster response data.
    data = resp.json()['data']
    # The cluster data columns.
    columns = data['tableHeaders']
    parsers_list = [parsers[col] for col in columns]
    # Parses a cluster content row.
    parse_row = lambda row: tuple(parsers_list[i](value) for i, value in enumerate(row))
    # The parsed cluster content list.
    content = map(parse_row, data['tableContent'])
    # Return the cluster data frames.
    return pd.DataFrame.from_records(content, index=index, columns=columns)

In [ ]:
def download_maf(cohort, out=None):
    """
    Downloads the MAF file for the given cancer type cohort.
    The default output file is `<cohort>.maf` in the
    sandbox directory.
    
    :param cohort: the cancer type cohort name
    :option out: the optional output file path
    :return: the output file path
    """
    # Download the MAF.
    url = 'http://firebrowse.org/api/v1/Analyses/Mutation/MAF'
    maf_file = out if out else os.path.join(sandbox, "%s.maf" % cohort) 
    print("Downloading the %s MAF file to %s..." % (cohort, maf_file))
    params = dict(format='tsv', cohort=cohort, page_size=2000)
    eof = False
    page = 1
    with open(maf_file, 'w') as f:
        while not eof:
            params['page'] = page
            resp = requests.get(url, params=params)
            text = resp.text
            if text:
                f.write(text)
                page = params['page'] = page + 1
            else:
                eof = True
            print('+', end='')
    print("MAF file downloaded.")
    return maf_file

In [ ]:
def build_network(maf_file, cutoff=.01):
    """
    Builds the network from the given MAF file.
    Only gene modules whose sample size exceeds a
    threshold are included. The threshold is the
    greater of 2 and sample count * cutoff, where
    sample count is the total number of MAF samples.
    
    :param maf_file: the downloaded MAF file
    :option cutoff: the minimum proportion of samples
    """
    # Clear the current Cytoscape session, if any.
    requests.delete(get_url('session')).ok
    # Read the MAF into a data frame.
    maf_df = pd.read_csv(maf_file, sep='\t')
    # The number of samples.
    sample_cnt = len({tsb for tsb in maf_df.Tumor_Sample_Barcode})
    print("Sample Count: %d" % sample_cnt)
    sample_cutoff = max(2, int(0.01*sample_cnt))
    print("Building the FI network with sample cut-off %d..." % sample_cutoff)
    # Build the FI network with 1% sample cut-off.
    body = dict(fiVersion='2016', format='MAF', file=maf_file,
                enteredGenes='', chooseHomoGenes=False,
                userLinkers=False, showUnLinked=False,
                fetchFIAnnotations=True,
                sampleCutoffValue=sample_cutoff)
    requests.post(get_fi_url('buildFISubNetwork'), json=body)
    print()
    print("The FI network is loaded to Cytoscape.")
    return maf_df

In [ ]:
# The cluster content value parsers.
cluster_parsers = {
    'Module': int, 'Nodes in Module': int, 'Node Percentage': float,
    'Samples in Module': int, 'Sample Percentage': float,
    'Node List': lambda s: s.split(',')
}

def cluster():
    """
    Cluster the network currently displayed in Cytoscape into
    gene modules.
    
    :return: a data frame with index *Module* and columns
      *module_size* and *genes*
    """
    # Cluster the currently displayed network.
    print("Clustering the FI network...")
    resp = requests.get(get_fi_url('cluster'))
    # Parse the response JSON into a data frame.
    parsed = parse_fi_table_response(resp, parsers=cluster_parsers, index='Module')
    # Rename the columns.
    renamed = parsed.rename(columns={'Node List': 'genes', 'Nodes in Module': 'module_size'})
    # Retain only the 'genes' and 'size' columns.
    sliced = renamed.loc[:, ['module_size', 'genes']]
    print("The cluster table is available in Cytoscape.")
    return sliced

In [ ]:
def prepare(cohort):
    """
    Prepares the given cancer type cohort for analysis.
    
    :param cohort: the cancer type cohort name
    :return: the clustered data frame
    """
    # Clear the current Cytoscape session, if any.
    try:
        requests.delete(get_url('session')).ok
    except requests.exceptions.ConnectionError:
        print("Error: Connection refused: is Cytoscape started?")
        raise
    maf_file = os.path.join(sandbox, "%s.maf" % cohort)
    # Download the MAF, if necessary.
    if (not os.path.exists(maf_file)):
        download_maf(cohort, out=maf_file)
    # Build the network.
    build_network(maf_file)
    # Cluster into gene modules.
    return cluster()

In [ ]:
def overlap_pvalue(N, gs1, gs2):
    """
    :param N: the number of background genes
    :param gs1: the first gene set
    :param gs2: the second gene set
    :return: the probability of gene set overlap from
      a background population of *N* genes
    """
    # The shared genes.
    overlap = set(gs1).intersection(set(gs2))
    M = len(overlap)
    n1 = len(gs1)
    n2 = len(gs2)
    # The CDF of complete overlap.
    kmax = min(n1, n2)
    cdf_max = hypergeom.cdf(kmax, N, n1, n2)
    # The CDF of the observed overlap.
    kmin = M - 1
    cdf_obs = hypergeom.cdf(kmin, N, n1, n2)
    # Return the overlap probability.
    return cdf_max - cdf_obs

In [ ]:
def filter_on_module_size(inputs, **kwargs):
    """
    Returns subsets of the given data frames with module
    count no greater than a threshold value. The threshold
    is given by the *max_len* option, with default the
    minimum module count of the input data frames.
    The module selection criterion is a gene module size
    which ensures that no more than *max_len* modules are
    selected from each data frame.
    
    :param input: the input {name: data frame} dictionary
    :return: the corresponding filtered data frames
    """
    # The size cutoff.
    max_len_opt = kwargs.get('max_len', None)
    n = max_len_opt or min(len(d) for df in modules)
    # The n largest data frames.
    largest = [df.module_size.nlargest(n) for df in inputs.values()]
    cutoff = max(ds.iloc[n - 1] for ds in largest)
    return {name: df.loc[df.module_size.ge(cutoff), :]
            for name, df in inputs.items()}

def slice_on_node_list(name, df):
    """
    Slices the given data frame vertically on the *genes* column.
    Renames the *Module* index to the given name and the *genes*
    column to '<name> Genes', e.g. 'BRCA Genes' for *name* 'BRCA'.
    
    :param name: the cancer type cohort name
    :param df: the gene modules data frame
    :return: the data frame with renamed index and column
    """
    rename_dict = {'genes': "%s Genes" % name}
    sliced = df.loc[:, ['genes']].rename(columns=rename_dict)
    sliced.index.names = [name]
    return sliced

def cartesian(df1, df2):
    """
    Takes the cartesian product of the input data frames.
    The resulting data frame has a mult-index with levels
    given by the respective input data frame indexes.
    
    :param df1: the first data frame
    :param df2: the second data frame
    :return: the cartesian product of the input data frames
    """
    rows = itertools.product(df1.iterrows(), df2.iterrows())
    values = (left.append(right) for (_, left), (_, right) in rows)
    indexes = [df1.index, df2.index]
    index_names = [index.name for index in indexes]
    index_values = [index.values for index in indexes]
    multi_index = pd.MultiIndex.from_product(index_values, names=index_names)
    return pd.DataFrame(values, index=multi_index)

In [ ]:
def append_overlap(n, cross):
    """
    Calculates the pair-wise overlap p-value for the given
    gene module cross-product.
    The return value is the cross-product augmented with
    two columns:
    * _uncorrected_pvalue_ - the hypergeometric test p value
    * _corrected_pvalue_ - the p-values corrected for
      multiple tests
    
    :param n: the background gene count
    :param cross: the cross-product data frame
    :return: the augmented data frame
    """
    # Calculates the overlap p-value.
    pvals = cross.agg(lambda row: overlap_pvalue(n, *row.values), axis=1)
    # Augment the cross-product with the uncorrrected p-values.
    overlap_df = cross.assign(uncorrected_pvalue=pvals)
    # Correct the given p-values for multiple comparison hypothesis
    # testing by applying the Benjamini–Hochberg FDR procedure.
    _, corrected, _, _ = multipletests(pvals.values, method='fdr_bh')
    overlap_df['corrected_pvalue'] = corrected
    return overlap_df

def unshared_series(df, shared_index):
    unshared_index = set(df.index.values) - set(shared_index)
    return df.loc[unshared_index, :].genes

def partition_shared(overlap_df, inputs, **kwargs):
    """
    Partitions the gene modules into three data series:
    * A shared data series for gene modules with significant
      overlap
    * Unshared BRCA data series
    * Unshared OV data series
    The shared data series has multi-index (`BRCA`, `OV`).
    The unshared data series have index `Module`.
    Each data series row has module gene set values.
    
    :param overlap_df: the input data frame is the BRCA x OV
      module cartesian cross-product augmented with a
      pair-wise overlap *corrected_pvalue*
    :param inputs: the {name: <data frame>} inputs
    :option cutoff: the corrected p-value cutoff value which
      determines a shared module (default .01)
    :return the {'shared': <series>, 'unshared': {name: <series>}}
      data series dictionary for each *name* in inputs
    """
    # The cut-off value.
    cutoff = kwargs.get('cutoff', .01)
    # The shared gene modules have corrected p-value < cutoff
    criterion = overlap_df['corrected_pvalue'].lt(cutoff)
    shared_df = overlap_df.loc[criterion]
    # TODO - partition unshared
    brca_only_ds = None
    ov_only_ds = None
    #brca_only_idx = overlap.index.droplevel('OV').difference(brca_shared.index).values
    #brca_df.loc[:, ['Nodes in Module', 'Node List']]
    brca_shared_genes = shared_df['BRCA Genes'].values
    ov_shared_genes = shared_df['OV Genes'].values
    merged_genes = [set(brca_shared_genes[i]) | set(ov_shared_genes[i])
                    for i in range(len(ov_shared_genes))]
    merged_df = shared_df.assign(merged_genes=merged_genes)
    shared_ds = merged_df.merged_genes
    shared_idx_values = [set(values)
                         for values in zip(*shared_ds.index.values)]
    shared_idx_names = shared_ds.index.names
    shared_idx_dict = {name: shared_idx_values[i]
                       for i, name in enumerate(shared_idx_names)}
    unshared = {name: unshared_series(df, shared_idx_dict[name])
                for name, df in inputs.items()}
    return dict(shared=shared_ds, unshared=unshared)

In [ ]:
def analyse_overlap(n, inputs, max_len=None):
    """
    Performs pair-wise overlap analysis on the given cluster
    data frames.
    
    :param n: the background gene count
    :param inputs: the {name: data frame} inputs dictionary
    :option max_len: option specifies an upper bound on the
      number of modules to be included in the overlap from
      each cancer type in the overlap (see filter_on_module_size) 
    :return the {'shared': <series>, 'unshared': {name: <series>}}
      data series dictionary for each *name* in inputs
    """
    # Select the largest modules.
    filtered = filter_on_module_size(inputs, max_len=max_len)
    # Prep pair-wise analysis by taking the cartesian product
    # of the two filtered data frames renamed with unique columns.
    sliced = {name: slice_on_node_list(name, df) for name, df in inputs.items()}
    cross_df = cartesian(*sliced.values())
    overlap_df = append_overlap(n, cross_df)
    return partition_shared(overlap_df, inputs)

In [ ]:
enrichment_parsers = {
    'ReactomePathway': lambda p: p,
    'RatioOfProteinInPathway': float,
    'NumberOfProteinInPathway': int,
    'ProteinFromGeneSet': int,
    'P-value': float,
    'FDR': float,
    'HitGenes': lambda s: s.split(',')
}

# Perform pathway enrichment analysis on the given gene list or set.
# Returns the result data frame.
def enrich(genes):
    data = ','.join(genes)
    # Perform the Reactome enrichment analysis.
    resp = requests.post(get_fi_url('ReactomePathwayEnrichment'), json=data)
    # Return the data frames.
    return parse_fi_table_response(resp, enrichment_parsers, index='ReactomePathway')

In [ ]:
# Prepare the BRCA cohort for analysis.
# This takes a while.
brca_df = prepare('BRCA')

*Expected Result*: Cytoscape displays the BRCA FI network
and clustered gene modules table.

In [ ]:
# Prepare the OV cohort for analysis.
# This takes a while.
ov_df = prepare('OV')

*Expected Result*: Cytoscape displays the OV FI network
and clustered gene modules table.

In [ ]:
# The number of background genes N is the Reactome FI
# network gene size.
BACKGROUND_GENE_CNT = 12283

# The analysis input.
inputs = {'BRCA': brca_df, 'OV': ov_df} 

In [ ]:
# Discover modules with significant overlap.
partition = analyse_overlap(BACKGROUND_GENE_CNT, inputs, max_len=20)
shared_ds = partition['shared']
brca_only_ds = partition['unshared']['BRCA']
ov_only_ds = partition['unshared']['OV']
# print(summary counts.)
print("Cluster BRCA module count: %d" % len(brca_df))
print("Cluster OV module count: %d" % len(ov_df))
print("Shared module pairs count: %d" % len(shared_ds))
print("Unshared BRCA module count: %d" % len(brca_only_ds))
print("Unshared OV module count: %d" % len(ov_only_ds))

In [ ]:
# Load Reactome pathways.
resp = requests.get(get_fi_url('pathwayTree'))
reactome_tree = resp.json()['data']

*Expected Result*: Reactome heirarchy displayed in the Cytoscape Control Panel.

In [ ]:
# Perform pathway enrichment on the shared gene modules.
enrichments = [enrich(gs) for gs in shared_ds.values]
# Select the pathway (as index) and FDR where FDR < .001
filtered = [e.FDR.loc[e.FDR.lt(.001)] for e in enrichments]
# Collect all significant shared pathways.
enriched = pd.concat(filtered).index.values
print("Significant enriched pathway count: %d" % enriched.size)

*Expected Result*: Enrichment results are successively displayed in the Cytoscape Table Panel.

In [ ]:
# TODO - Perform pathway enrichment on the unshared BRCA gene modules.


In [ ]:
# TODO - Export the diagram for one pathway.

In [ ]:
# TODO - Move doc cells below to appropriate steps above.

(cf. User Guide
[Explore Pathways](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Explore_Reactome_Pathways)
step)

(cf. User Guide
[Enrichment Analysis](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Pathway_Enrichment_Analysis)
step)

(cf. User Guide
[Mutation Analysis](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Gene_Set.2FMutation_Analysis)
step)